In [ ]:
import os
import time
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import gym
from gym import spaces
import math
import pandas as pd
from gym import wrappers
from collections import deque
from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool` is a deprecated alias')

In [ ]:
import pandas as pd 
from collections import deque
import gym

import os 
import numpy as np
from random import sample
import torch as T 
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
import torch.multiprocessing as mp
from torch.distributions import Categorical,Normal,MultivariateNormal
from multiprocessing.pool import Pool
import random
from rltorch.network import create_linear_network,BaseNetwork
import torch
from torch.optim import Adam
# torch.set_grad_enabled(True)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device="cpu"

In [ ]:
#Parameters
num_iotds = 100 #number
X_min = 0 #meters
X_max = 200 #meters
Y_min = 0 #meters
Y_max = 200 #meters
O = 9 #number
R_obstacles_max = 10 #meters
Safe_radius = 2 #meters
Cluster_radius = 20 #meters
H = 50 #meters
E_wake = 0.1 #Watt/s
Comm_radius = 10 #meters
Wake_radius = 30 #meters
B = 500 * 10**6 # hertz , available channel bandwidth
beta_o = 31.62 # Watts, path loss at reference distance
rho_m = 0.1 # Watts, transmitting power of a CH
sigma = 10**-13 # Watts , noise power
alpha = 2 # path loss exponent
V_max = 10 #m/s
V_min = 5 #m/s
E_start_UAV = 1000 #Joules
E_hover = 100 #Watt
U_tip = 200 #m/s
v_o = 10 # m/s , mean rotor induced velocity in the hovering state
rho = 1.225 # kg/m^3, air density
delta = 0.202 # rotor area
d_o = 0.1 # fusealage drag ratio , dimensionless
s = 0.1 # rotor solidity , dimensionless
P_o = 0.7 # blade profile power coefficient , dimensionless
P_i = 0.2 # induced power coefficient in the hovering state 
P_h = 100 #Watt, power required for hovering
D_max = 10 # meters
D_min = 1 # meters
E_IoTD_start = 1000000 #Joules
Distance_O_penalty = 100
Distance_multiplier = 0.1
h_reward = 100000

k_E=0.1
k_AOI=0.1

In [ ]:
import random
from math import sqrt

def calculate_distance(coord1, coord2):
    distance = sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)
    return distance

def calculate_UAV_CH_distance(P_t,P_m):
    distance = sqrt((P_t[0] - P_m[0])**2 + (P_t[1] - P_m[1])**2 + P_t[2]**2)
    return distance

def calculate_power_gain_UAV_CH(beta_o,P_t,P_m):
    distance = calculate_UAV_CH_distance(P_t,P_m)
    power_gain = beta_o/(distance**2)
    return power_gain

def calculate_UAV_CH_transmission_rate(B, P_t, P_m, beta_o, sigma, rho_m):
    power_gain = calculate_power_gain_UAV_CH(beta_o, P_t, P_m)
    transmission_rate = B * np.log2(1 + (rho_m * power_gain)/(sigma**2))
    return transmission_rate

def calculate_UAV_CH_Upload_time(B, P_t, P_m, beta_o, sigma, rho_m, I_m):
    transmission_rate = calculate_UAV_CH_transmission_rate(B, P_t, P_m, beta_o, B , sigma, rho_m)
    upload_time = I_m/transmission_rate
    return upload_time

def calculate_CH_IoTD_distance(P_m, P_n):
    distance = sqrt((P_m[0] - P_n[0])**2 + (P_m[1] - P_n[1])**2)
    return distance

def calculate_CH_IOTD_transmission_rate(B, P_m, P_n, beta_m,rho_n,alpha , rho_i):
    distance = calculate_CH_IoTD_distance(P_m, P_n)
    power_gain = beta_m/(distance**alpha)
    transmission_rate = B * np.log2(1 + (rho_i * power_gain)/(rho_n))
    return transmission_rate

def calculate_UAV_Obstacle_distance(P_t,L_o):
    distance = sqrt((P_t[0]-L_o[0])**2 + (P_t[1]-L_o[1])**2) - L_o[2]
    return distance

def calculate_UAV_travel_energy_per_second(P_o,v_t,U_tip,P_i,v_o,d_o,rho,s,delta):
    energy_per_second = P_o*(1+3*(v_t**2)/(U_tip**2)) + P_i*v_o/v_t + d_o*rho*s*delta*(v_t**3)/2
    return energy_per_second

def maximum_minimum_distance_clustering(nodes, cluster_radius):
    cluster_heads = []
    remaining_nodes = nodes.copy()

    first_node=remaining_nodes.pop(0)
    
    cluster_heads.append((first_node[0],first_node[1]))

    while remaining_nodes:
        max_min_distance = 0
        max_min_distance_node = None
        for node in remaining_nodes:
            min_distance = min(calculate_distance(node, ch) for ch in cluster_heads)
            if min_distance > max_min_distance and min_distance > cluster_radius:
                max_min_distance = min_distance
                max_min_distance_node = node
        if max_min_distance_node is None:
            break
        cluster_heads.append((max_min_distance_node[0],max_min_distance_node[1]))
        remaining_nodes.remove(max_min_distance_node)

    # print(cluster_heads)
    clusters = {ch: [] for ch in cluster_heads}
    for node in remaining_nodes:
        nearest_ch = min(cluster_heads, key=lambda ch: calculate_distance(node, ch))
        clusters[nearest_ch].append(node)

    return clusters

def generate_IoTD_coordinates_outside_obstacles(num_coordinates, Obstacle_coordinates):
    coordinates = []
    xmin = X_min
    xmax = X_max
    ymin = Y_min
    ymax = Y_max
    for i in range(num_coordinates):
        x = random.uniform(xmin, xmax)
        y = random.uniform(ymin, ymax)
        new_coord = (x, y)
        if not any(calculate_distance(new_coord, obstacle) <= obstacle[2] + Safe_radius for obstacle in Obstacle_coordinates):
            coordinates.append(new_coord)
    
    return coordinates

    

def generate_Obstacle_coordinates(num_coordinates):
    coordinates = []
    xmin = X_min
    xmax = X_max
    ymin = Y_min
    ymax = Y_max
    for i in range(num_coordinates):
        x = random.uniform(xmin, xmax)
        y = random.uniform(ymin, ymax)
        r = random.uniform(0, R_obstacles_max)
        new_coord = (x, y, r)
        coordinates.append(new_coord)
    
    return coordinates


In [ ]:
import csv
import random
from math import sqrt

coordinates_file_name="coordinates.csv"


def calculate_distance(coord1, coord2):
    distance = sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)
#     print(distance)
    return distance

def save_to_csv(filename, coordinates):
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['X', 'Y'])
        csv_writer.writerows(coordinates)

X=[(random.randint(X_min,X_max),random.randint(Y_min,Y_max)) for _ in range(num_iotds)]
print(X)
save_to_csv(coordinates_file_name, X)

print(f"{num_iotds} coordinates generated and saved to {coordinates_file_name}.")
def distance(u,v,w,x,y,z):
    distance = ((u-x)**2 + (v-y)**2 + (w-z)**2)**0.5
    return distance

In [ ]:
O_obstacles = [
    [50,50,10],
    [100,50,10],
    [150,50,10],
    [50,100,10],
    [100,100,10],
    [150,100,10],
    [50,150,10],
    [100,150,10],
    [150,150,10]
]
N = generate_IoTD_coordinates_outside_obstacles(100, O_obstacles)
# N=[
#     [25,75],
#     [25,125],
#     [75,75],
#     [75,125],
#     [125,75],
#     [125,125],
#     [175,75],
#     [175,125],
#     [75,25],
#     [125,25],
#     [75,175],
#     [125,175],
    
# ] 
import matplotlib.pyplot as plt

# Plot IoTDs
# for coord in N:
#     plt.scatter(coord[0], coord[1], color='blue', label='IoTD')
    
# Plot Obstacles
# for obstacle in O_obstacles:
#     circle = plt.Circle((obstacle[0], obstacle[1]), obstacle[2], color='red', fill=False)
#     plt.gca().add_patch(circle)

# Set plot limits
# plt.xlim(X_min - 10, X_max + 10)
# plt.ylim(Y_min - 10, Y_max + 10)

# Add legend
# plt.legend()
# plt.gca().set_aspect('equal')

# Show the plot
# plt.show()


In [ ]:
X=N
Clusters = maximum_minimum_distance_clustering(X, Cluster_radius)
X=list(Clusters.keys())
print(X)
M = []

for cluster_head, cluster_nodes in Clusters.items():
    M.append(cluster_head)

M=X.copy()

x,y=[node[0] for node in Clusters.keys()],[node[1] for node in Clusters.keys()]
for i in Clusters.keys():
    color=(random.choice(range(256))/255,
           random.choice(range(256))/255,
           random.choice(range(256))/255)
    plt.scatter(i[0],i[1],color=color,marker='*')
    for j in Clusters[i]:
        plt.scatter(j[0],j[1],color=color)
for obstacle in O_obstacles:
    circle = plt.Circle((obstacle[0], obstacle[1]), obstacle[2], color='red', fill=False)
    plt.gca().add_patch(circle)
plt.gca().set_aspect('equal')
plt.show()
print(len(x))
Data = []

for cluster_head , cluster_nodes in Clusters.items():
    Data_cluster = 0
    for node in cluster_nodes:
        Data_cluster += random.randint(1, 10)
    Data.append(Data_cluster)

In [ ]:
class Multi_Agent_UAV(gym.Env):
    def __init__(self, initial_UAV_state = None):
        super(Multi_Agent_UAV, self).__init__()

        self.agent = 2
        self.pos_list = []
        self.color = ['red', 'green', 'blue', 'cyan', 'magenta']
        self.initial_UAV_state = np.array([100,100])
        self.UE_count = len(X)
        self.Z = 20
        self.max_angle = 2 * math.pi
        self.phi_n = np.radians(42.44)  # in degrees
        self.C_max_t = (self.Z / np.tan(self.phi_n))
        self.boundary_x = self.boundary_y = 200.
        self.max_episode_steps = 30 # Maximum number of steps in a single episode, after which environment returns done = True
        self._max_episode_steps = 30
        self.current_episode_timestep = 0
        self.done = False
        self.visited = np.array([0 for i in range(self.UE_count)])
        self.state_space_lb = np.array([0, 0], dtype = np.float32)
        self.state_space_ub = np.array([self.boundary_x, self.boundary_y], dtype = np.float32)
        self.energy_array=[]
        self.AOI_array=[]
        
        # plotting parameters
        self.plotting_boundary_buffer = 5.
        self.UAV_coverage_circle_color = 'lavender'
        self.UAV_path_color = 'green'
        obh = []
        obl = []
        self.weight = []
        self.visited = np.array([0.0 for i in range(self.UE_count)])
        
        # AOI
        # obh.append(float('inf'))
        # obl.append(0)
        
        # UAV Energy
        # obh.append(E_start_UAV)
        # obl.append(0)
        for i in range(len(X)+6+self.UE_count):
            obh.append(float('inf')) 
        for i in range(len(X)+6+self.UE_count):
            obl.append(float('-inf'))
        
        
        high = np.array(obh, dtype=np.float32)
        low = np.array(obl, dtype=np.float32)
        high1 = np.array([len(X), 180.0, 6.0], dtype=np.float32)
        low1 = np.array([0, -180.0, 5.0], dtype=np.float32)
        
        self.observation_space = np.array([spaces.Box(low, high) for i in range(self.agent)])
        #self.action_space = spaces.Box(low=low1, high=high1, dtype=np.float32)
        self.action_space = spaces.Box(-1, 1, (2,), dtype=np.float32)

        self.place_UEs(position="random")


    def reset(self):
        # RANDOMLY SELECTS UAV STARTING POSITION
        self.current_state = self.select_random_state()
        ground_UAV_state = self.current_state[:4]
        self.visited = np.array([0 for i in range(self.UE_count)])
        self.timesteps_in_episode = 0
        self.done = False
        self.c_state = []
        self.AOI=[0 for _ in range(self.UE_count)]
        self.UAV_energy=[E_start_UAV for _ in range(self.agent)]
        for j in range(self.agent):
            obs = []
            obs.append(j)
            obs.append(self.current_state[j][0].copy()/1000)
            obs.append(self.current_state[j][1].copy()/1000)
            obs.append(self.current_state[j][0].copy()/1000)
            obs.append(self.current_state[j][1].copy()/1000)
            for i in range(len(X)):
                obs.append(-(math.dist(self.UE_positions[i], ground_UAV_state[j]))/1000)
            obs.append(self.UAV_energy[j]/2000)
            obs+=[i/200 for i in self.AOI]
            self.c_state.append(obs)
        self.c_state = np.array(self.c_state)
        return self.c_state
    
    
    def get_count_of_UEs_covered(self):
        profit = 0
        ground_UAV_state = self.current_state[:self.agent]
        for j in range(self.agent):
            for i in range(self.UE_count):
                if(math.dist(self.UE_positions[i], ground_UAV_state[j])<=self.C_max_t):
                    if(self.visited[i]==0):
                        profit = profit + 500
                        self.visited[i] =1
                        self.AOI[i]=0
                    else:
                        profit = profit-100
                        self.AOI[i]+=1
                else:
                    if(self.visited[i]==0):
                        profit = profit -1
                        self.AOI[i]+=1
#         horizontal_dist_UE_UAV = np.linalg.norm(self.UE_positions - ground_UAV_state, axis = 1)
        # print("horizontal distance b/w UEs and UAV: ", horizontal_dist_UE_UAV)
#         rho_array = (horizontal_dist_UE_UAV <= (self.C_max_t)) * 1  # binary association vector
#         M_t = rho_array.sum()
        return profit


    def render(self, **kwargs):
        '''
        Function to visualize UAV position
        (this function has different configurations for different use cases)
        '''
        if 'UAV_positions_list' in kwargs:
            self.render_UAV_movement_through_episode(UAV_positions_list = kwargs['UAV_positions_list'])

        if 'fig' not in kwargs:
            fig, ax = plt.subplots(1, 1, figsize=(20,10))
            ax = np.array([ax])
            kwargs['fig'] = fig
            kwargs['ax'] = ax
            kwargs['i'] = 0
        
        if 'reward' not in kwargs:
            kwargs['reward'] = 'Not known'
        self.render_position_plot(**kwargs)


    def step(self, action: np.ndarray):
        '''
        Takes action moving environment from current_state to next_state
        Arguments: `action` to be taken
        Returns: new_state, reward, done, info(=None)
        '''
        # action = action.numpy().flatten()
#         err_msg = f"{action!r} ({type(action)}) invalid"
#         assert self.action_space.contains(action), err_msg
#         assert self.current_state is not None, "Call reset before using step method."
        self.timesteps_in_episode += 1
        prev_UAV_state = self.current_state.copy()
        _, inside_bounds = self.move(action)
        
        # print(f"prev state : {prev_UAV_state} new state : {self.current_state}")
        energy_used=[0 for _ in range(self.agent)]
        for i in range(self.agent):
            velocity = (action[i][0]+1.5)*5 + 2.5  # velocity of the UAV
            velocity = v_o
            step_size = (action[i][0]+1.2)*10    # step size of the UAV
            angle = (action[i][1])*np.pi  # angle of the UAV
            
            time_flew = calculate_distance(prev_UAV_state[i], self.current_state[i])/velocity
            # Calculating the energy used by the UAV in the current step
            energy_used[i] = time_flew * calculate_UAV_travel_energy_per_second(P_o, velocity, U_tip, P_i, v_o, d_o, rho, s, delta)
            
            self.UAV_energy[i]-=energy_used[i]
        
        
        # print(type(self.current_state), type(prev_UAV_state))
#         if not inside_bounds:
#             MENTIONED OUTPUT IS OUT OF BOUNDS
#             self.is_done(end = True)
#             reward = -10000
#             return self.current_state, reward, self.done, None

#         ground_UAV_state = self.current_state[:2]
#         horizontal_dist_UE_UAV = np.linalg.norm(self.UE_positions - ground_UAV_state, axis = 1)
#         rho_array = (horizontal_dist_UE_UAV <= (self.C_max_t)) * 1  # binary association vector
#         # self.current_state = np.concatenate((self.current_state[:2], rho_array))
#         M_t = rho_array.sum()  # no. of UEs served by the agent

        reward = self.get_count_of_UEs_covered()
        
        # print(f"AOI : {self.AOI}")
        
        for i in range(self.agent):
            if self.UAV_energy[i]<0:
                reward-=10000
            else:
                reward-=k_E*energy_used[i]
        avg_AOI=sum(self.AOI)/len(self.AOI)
        reward-=k_AOI*avg_AOI
        # reward-=50
        
        # for i in range(self.UE_count):
        #     reward-=k_AOI*self.AOI[i]
        
#         if(sum(self.visited)>15):
#             reward = 100 * sum(self.visited) 
        self.is_done(sum(self.visited))
        if self.is_done:
            self.energy_array.append(self.UAV_energy)
            self.AOI_array.append(avg_AOI)
        self.c_state=[]
        ground_UAV_state = self.current_state[:self.agent]
        for j in range(self.agent):
            obs = []
            obs.append(j)
            obs.append(self.current_state[j][0].copy()/1000)
            obs.append(self.current_state[j][1].copy()/1000)
            obs.append(prev_UAV_state[j][0].copy()/1000)
            obs.append(prev_UAV_state[j][1].copy()/1000)
            for i in range(len(X)):
                if(self.visited[i] == 0):
                    obs.append(-(math.dist(self.UE_positions[i], ground_UAV_state[j]))/10000)
                else:
                    obs.append(1)
            obs.append(self.UAV_energy[j]/2000)
            obs+=[i/200 for i in self.AOI]
            self.c_state.append(obs)
        self.c_state = np.array(self.c_state)
        for i in range(self.agent):
            for j in range(self.agent):
                if(i!=j):
                    dis = distance(self.current_state[i][0],self.current_state[i][1], self.current_state[j][0], self.current_state[j][1],0,0)
                    if(dis<10):
                        reward = reward -10000
        return self.c_state, reward, self.done, None
    

    def move(self, action: np.ndarray):
        '''
        Helper function to step() function.
        Clips the passed action to fit within action space bounds.
        Calculates new state after performing the passed action, and updates UAV position accordingly. 
        '''
#         evaluates new state reached upon performing the move and saves it in self.current_state
        # ACTION = [dx, dy]
#         angle1= math.degrees(math.atan(action))
#         print(angle1)
        for i in range(self.agent):
            self.action_step_size = (action[i][0]+1.5)*10
            angle1 = action[i][1] * 180
            angle = angle1*math.pi/180
            x_next = self.current_state[i][0] + (math.cos(angle) * self.action_step_size)
            y_next = self.current_state[i][1] + (math.sin(angle) * self.action_step_size)
            # updating horizontal_direction_angle if the new move is out of boundary
            if((x_next < 0) or (x_next > self.boundary_x) or (y_next < 0) or (y_next > self.boundary_y)):
            # UAV REMAINS IN ITS CURRENT POSITION
                return action, False

            self.current_state[i][:2] = np.array([x_next, y_next])
        return action, True


    def is_done(self, M_t=0, end=False):
        '''
        Helper function to check if episode needs to be terminated
        '''
        if end == True:
            self.done = True
        elif(sum(self.visited)==len(X)):
            print(sum(self.visited))
            self.done = True
        elif(self.timesteps_in_episode >= self.max_episode_steps):
            self.done = True
        return

    
    def select_random_state(self):
        '''
        Selects (and returns) random initial state (within bounds) for the UAV
        '''
        UAV_pos_list = []
        for _ in range(self.agent):         
            new_x = 100.0
            new_y = 100.0

            UAV_pos_list.append(np.array([new_x, new_y]))

        return np.array(UAV_pos_list)

    def place_UEs_randomly(self, random_UE_count = None):
        '''
        Helper function to place_UEs
        places `random_UE_count` UEs randomly onto the rectangular region
        if kwargs has the key `exclude_center`, then the circular region spanned by `exclude_center` \
        and `exclude_radius` is excluded
        '''

        if random_UE_count is None:
          random_UE_count = self.UE_count

        # places UE_count UEs on grid randomly
        if type(self.UE_positions) == np.ndarray:
            self.UE_positions = self.UE_positions.tolist()

        randomly_placed_count = 0
        while(randomly_placed_count < random_UE_count):
            x = X[randomly_placed_count][0]
            y = X[randomly_placed_count][1]
            coords = np.array([x, y])
            self.UE_positions.append(coords)
            randomly_placed_count += 1
        

        self.UE_positions = np.array(self.UE_positions)


    def place_UEs(self, position="random"):
        '''
        Function to place UEs onto the rectangular region
        '''
        self.UE_positions = np.zeros((0, 2))
        self.place_UEs_randomly(self.UE_count)

        # saves in self.UE_positions
        self.UE_positions = np.array(self.UE_positions)
        return
    def render_UAV_movement_through_episode(self, **kwargs):
        '''
        Helper function to visualize UAV movement through an episode
        Arguments: A list specifying UAV positions throughout the episode
        '''

        if 'reward' not in kwargs:
            kwargs['reward'] = 'Not known'
        
        buffer = 5.
        fig, ax = plt.subplots(1, figsize=(10,10))
        ax.set_xlim(-buffer, self.boundary_x+buffer)
        ax.set_ylim(-buffer, self.boundary_y+buffer)
        ax.grid()
        
        for j in range(len(X)):
            UE_x1 = X[j][0]
            UE_y1 = X[j][1]
            if(self.visited[j] == 0):
                ax.plot(UE_x1, UE_y1, color=self.color[0], marker='o', markersize=6, linestyle = '') 
            else:
                ax.plot(UE_x1, UE_y1, color=self.color[1], marker='o', markersize=6, linestyle = '')
        
        for i in range(self.agent):
            ax.plot(kwargs['UAV_positions_list'][:, i, 0] , kwargs['UAV_positions_list'][:, i, 1], color=self.UAV_path_color, markersize=6, linestyle = '-') #, label = "UAV Path Color")
            ax.plot(kwargs['UAV_positions_list'][-1, i, 0], kwargs['UAV_positions_list'][-1, i, 1], color='blue', marker='x', markersize=9, linestyle = '') #, label = "UAV ending position")
            
            UAV_coverage_area = plt.Circle((kwargs['UAV_positions_list'][-1, i, 0], kwargs['UAV_positions_list'][-1, i, 1]), self.C_max_t, color = self.UAV_coverage_circle_color)
            ax.add_artist(UAV_coverage_area)

        ax.set_aspect(1)
        # naming the x axis
        ax.set_xlabel('X pos (m)')
        # naming the y axis
        ax.set_ylabel('Y pos (m)')
        ax.set_title('UAV movement through episode')
        # giving a title to my graph
        # plt.title('Visually Appealing!')

        # show a legend on the plot
        ax.legend()
        # plt.show()
        return
    
    def render_position_plot(self, **kwargs):
        '''
        Helper function to render(), plots the current position plot on given axes. 
        Plotting position plot of UAV's current position and coverage
        '''
        ax = kwargs['ax']
        plot_number = kwargs['i']
        idx = 0
        reward = kwargs['reward']

        buffer = 5.
        ax[plot_number].set_xlim(-buffer, self.boundary_x+buffer)
        ax[plot_number].set_ylim(-buffer, self.boundary_y+buffer)
        ax[plot_number].grid()
        for j in range(len(X)):
            UE_x1 = X[j][0]
            UE_y1 = X[j][1]
            if(self.visited[j] ==0):
                ax[plot_number].plot(UE_x1, UE_y1, color=self.color[0], marker='o', markersize=6, linestyle = '')
            else:
                ax[plot_number].plot(UE_x1, UE_y1, color=self.color[1], marker='o', markersize=6, linestyle = '')
         
        
        for i in range(self.agent):
            ax[plot_number].plot(self.current_state[i][0], self.current_state[i][1], color='blue', marker='x', markersize=9, linestyle = '', label = "UAV")
            UAV_coverage_area = plt.Circle((self.current_state[i][0] , self.current_state[i][1] ), self.C_max_t, color = self.UAV_coverage_circle_color)
            ax[plot_number].add_artist(UAV_coverage_area)
      

        if 'learnt_policy_visualization' in kwargs:
            position_action_list = kwargs['position_action_list']
            for position_action in position_action_list:
                position = position_action[0]
                action = position_action[1]
                action = np.clip(action, self.action_space_lb, self.action_space_ub)
                action = (((self.action_space_coversion_ub - self.action_space_coversion_lb) * (action + 1)) / 2) + self.action_space_coversion_lb
                print("scaled action: ", action)
                # print("action after scaling: ", action)
                new_state = self.move(action, get_new_state=True, provided_center=position)
                print("previous state: ", position, ", new state: ", new_state)
                # print("new_state: ", new_state)
                ax[plot_number].plot(position[0], position[1], color='blue', marker='o', markersize=6, linestyle = '')
                ax[plot_number].plot(new_state[0], new_state[1], color='orange', marker='o', markersize=6, linestyle = '')
                ax[plot_number].arrow(position[0], position[1], new_state[0] - position[0], new_state[1] - position[1], head_width=0.2, head_length=0.1)


        ax[plot_number].set_aspect(1)
        # naming the x axis
        ax[plot_number].set_xlabel('X pos (m)')
        # naming the y axis
        ax[plot_number].set_ylabel('Y pos (m)')
        ax[plot_number].set_title('reward: ' + str(reward))
        # giving a title to my graph
        # plt.title('Visually Appealing!')

        # show a legend on the plot
        ax[plot_number].legend()
        # plt.show()

In [ ]:
env = Multi_Agent_UAV()
state = env.reset()
UAV_positions_list = []
ab=[]
for i in range(env.agent):
    k = []
    k.append(env.current_state[i][0])
    k.append(env.current_state[i][1])
    ab.append(k)
UAV_positions_list.append(ab)
a = [env.action_space.sample() for i in range(env.agent)]
# print(a)
next_state, reward, done, _ = env.step(a)
print(next_state, reward)
ab=[]
for i in range(env.agent):
    k = []
    k.append(env.current_state[i][0])
    k.append(env.current_state[i][1])
    ab.append(k)
UAV_positions_list.append(ab)
b = [env.action_space.sample() for i in range(env.agent)]
# print(b)
next_state, reward, done, _ = env.step(b)
# print(next_state, reward)
ab=[]
for i in range(env.agent):
    k = []
    k.append(env.current_state[i][0])
    k.append(env.current_state[i][1])
    ab.append(k)
UAV_positions_list.append(ab)
UAV_positions_list = np.array(UAV_positions_list)
print("List",UAV_positions_list)
env.render_UAV_movement_through_episode(UAV_positions_list = UAV_positions_list)

In [ ]:
max_episodes = 10000
import copy
       
action_std = 0.6                    
action_std_decay_rate = 0.05       
min_action_std = 0.1                

  
K_epochs = 30        

eps_clip = 0.2         
gamma = 0.99           
max_steps=50
update=max_steps*4
overallsteps=200000
lr_actor = 0.0003      
lr_critic = 0.001      
string='Multi_Agent_UAV'

In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import deque
import math

# Uncommenting this for actual implementation
# device = T.device("cuda" if T.cuda.is_available() else "cpu")
device = T.device("cpu")  # Placeholder for the code review

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
        
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = zip(*batch)
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)
        
    def clear(self):
        self.buffer.clear()

class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, action_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

class DoubleDQNAgent:
    def __init__(self, envstr, learning_rate=0.0003, gamma=0.99, epsilon_start=1.0, 
                 epsilon_end=0.01, epsilon_decay=0.995, buffer_size=100000, 
                 batch_size=64, target_update=100):
        env = Multi_Agent_UAV()
        
        # Define the number of continuous action dimensions
        self.action_dims = 2  # Two dimensions: speed and angle
        # Define the number of discrete divisions per dimension
        self.divisions_per_dim = 5  # 5 discrete values per dimension
        
        # Total number of discrete actions (combinations)
        self.action_dim = self.divisions_per_dim ** self.action_dims
        self.state_dim = env.observation_space[0].shape[0]
        
        # Q Networks
        self.q_network = QNetwork(self.state_dim, self.action_dim).to(device)
        self.target_network = QNetwork(self.state_dim, self.action_dim).to(device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()
        
        # Create action mapping (discrete index to continuous vector)
        self.action_space = []
        action_values = np.linspace(-1, 1, self.divisions_per_dim)
        
        # Create all combinations of actions
        for i in range(self.divisions_per_dim):
            for j in range(self.divisions_per_dim):
                self.action_space.append([action_values[i], action_values[j]])
        
        # Hyperparameters
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.target_update = target_update
        
        # Optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        
        # Replay buffer
        self.buffer = ReplayBuffer(buffer_size)
        
        # Counter for target network updates
        self.update_counter = 0
    
    def get_action(self, state):
        if np.random.random() < self.epsilon:
            # Random action
            action_index = np.random.randint(0, self.action_dim)
            # Map to continuous action vector
            return self.action_space[action_index]
        
        with T.no_grad():
            state = T.FloatTensor(np.array(state).reshape(1, -1)).to(device)
            q_values = self.q_network(state)
            action_index = T.argmax(q_values, dim=1).item()
            # Map to continuous action vector
            return self.action_space[action_index]
    
    def update(self):
        if len(self.buffer) < self.batch_size:
            return
        
        # Sample a batch from replay buffer
        states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
        
        # Convert to tensors
        states = T.FloatTensor(np.array(states)).to(device)
        actions = T.LongTensor(np.array(actions)).to(device)
        rewards = T.FloatTensor(np.array(rewards)).to(device)
        next_states = T.FloatTensor(np.array(next_states)).to(device)
        dones = T.FloatTensor(np.array(dones)).to(device)
        
        # Get Q values for current states and actions
        q_values = self.q_network(states).gather(1, actions.unsqueeze(1))
        
        # Double DQN: Use online network to select actions and target network to evaluate them
        with T.no_grad():
            # Get actions that would be taken by online network
            online_next_q_values = self.q_network(next_states)
            online_next_actions = T.argmax(online_next_q_values, dim=1, keepdim=True)
            
            # Evaluate those actions using the target network
            next_q_values = self.target_network(next_states).gather(1, online_next_actions)
            target_q_values = rewards.unsqueeze(1) + (1 - dones.unsqueeze(1)) * self.gamma * next_q_values
        
        # Compute loss
        loss = F.mse_loss(q_values, target_q_values)
        
        # Update Q network
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping similar to original code
        for p in self.q_network.modules():
            T.nn.utils.clip_grad_norm_(p.parameters(), 1.0)
        self.optimizer.step()
        
        # Update target network
        self.update_counter += 1
        if self.update_counter % self.target_update == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())
    
    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)
    
    def save_experience(self, state, action, reward, next_state, done):
        # Find the closest discrete action in our action space
        action_idx = self.get_closest_action_index(action)
        self.buffer.push(state, action_idx, reward, next_state, done)
    
    def get_closest_action_index(self, action):
        # Find the index of the closest action in the discrete action space
        distances = [np.sum((np.array(a) - np.array(action))**2) for a in self.action_space]
        return np.argmin(distances)

# Training loop
def train_double_dqn(env, agents, max_episodes=1000, max_steps=500):
    step = 0
    rp = 0
    scores = []
    
    while rp <= max_episodes:
        interscore = 0
        
        state = env.reset()
        done = False
        
        for j in range(max_steps):
            step += 1
            
            # Get actions from agents
            actions = []
            for i in range(env.agent):
                action = agents[i].get_action(state[i])
                actions.append(action)
            
            # Take step in environment
            next_state, reward, done, _ = env.step(actions)
            interscore += reward
            
            # Store experience in each agent's buffer
            for i in range(env.agent):
                agents[i].save_experience(state[i], actions[i], reward, next_state[i], done)
                
                # Update Double DQN
                if step % 4 == 0:  # Update frequency
                    agents[i].update()
                
                # Decay epsilon
                if step % 1000 == 0:
                    agents[i].decay_epsilon()
            
            state = next_state
            scores.append(interscore)
            
            if done:
                rp += 1
                break
        
        if rp % 40 == 0:
            print(np.mean(scores[-100:]), step, " episodes : ", rp)
    
    return scores

# Initialize environment and agents
env = Multi_Agent_UAV()
agents = [DoubleDQNAgent(
    envstr="Multi_Agent_UAV",
    learning_rate=0.0001,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.3,
    epsilon_decay=0.999,
    buffer_size=1000,
    batch_size=32,
    target_update=1000
) for _ in range(env.agent)]

# Train agents
scores = train_double_dqn(env, agents, max_episodes=max_episodes, max_steps=max_steps)

In [ ]:
def visualize_trained_agent(env, agents):
    state = env.reset()
    done = False
    score = 0
    UAV_positions_list = []
    
    while not done:
        # Get actions from all agents using the trained DQN
        actions = []
        for i in range(env.agent):
            # Get action without exploration (epsilon=0)
            with T.no_grad():
                state_tensor = T.FloatTensor(np.array(state[i]).reshape(1, -1)).to(device)
                q_values = agents[i].q_network(state_tensor)
                action_index = T.argmax(q_values, dim=1).item()
                action = agents[i].action_space[action_index]
            actions.append(action)
        
        # Take step in environment
        state, reward, done, _ = env.step(actions)
        score += reward
        
        # Record UAV positions
        ab = []
        for i in range(env.agent):
            k = []
            k.append(env.current_state[i][0])
            k.append(env.current_state[i][1])
            ab.append(k)
        UAV_positions_list.append(ab)
        
        print('score %.1f' % score, 'avg_score %.1f' % score)
    
    # Convert to numpy array for rendering
    UAV_positions_list = np.array(UAV_positions_list)
    
    # Render the UAV movement
    env.render_UAV_movement_through_episode(UAV_positions_list=UAV_positions_list)
    
    return score

# After training is complete, visualize the trained agents
final_score = visualize_trained_agent(env, agents)
print(f"Final visualization score: {final_score}")

In [ ]:
df=pd.DataFrame(data={'scores':scores})
df2=df.copy()
df2['max']=df2.scores.rolling(100).mean()
df2.drop(columns=['scores'],inplace=True)

df['mean']=df.scores.rolling(100).mean()
df

In [ ]:
df.plot(figsize=(18,10))

In [ ]:
def calculate_rolling_mean(a,shift):
    cur_mean=np.mean(a[:shift])
    result=[cur_mean]
    
    
    for i in range(len(a)-shift):
        cur_mean+=(a[i+shift]-a[i])/shift
        result.append(cur_mean)
    return result

In [ ]:
utility=calculate_rolling_mean(list(df2.loc[:,'max'].dropna()),500)
plt.plot(range(len(utility)),utility)
plt.xlabel("iterations")
plt.ylabel("avg. utility")
plt.show()

In [ ]:
energy_plot=[np.mean(i) for i in env.energy_array]
shift=500
energy_plot=calculate_rolling_mean(energy_plot,shift)
plt.plot(range(len(energy_plot)),energy_plot)
plt.xlabel("iterations")
plt.ylabel("avg. energy")
plt.show()

In [ ]:
aoi_plot=[np.mean(i) for i in env.AOI_array]
shift=500
aoi_plot=calculate_rolling_mean(aoi_plot,shift)
plt.plot(range(len(aoi_plot)),aoi_plot)
plt.xlabel("iterations")
plt.ylabel("avg. AOI")
plt.show()

In [ ]:
import json
result_count="doubleDQN"
save_to_file=True
if save_to_file:
    with open(f"./results/result_{result_count}_utility.json","w") as f:
        f.write(json.dumps(utility))
    with open(f"./results/result_{result_count}_energy.json","w") as f:
        f.write(json.dumps(env.energy_array))
    with open(f"./results/result_{result_count}_AOI.json","w") as f:
        f.write(json.dumps(env.AOI_array))